# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [9]:
import nltk #library for natural language toolkit
import pandas as pd
# nltk.download('punkt') ## run in case of punkt and puknkt_tab error when calculcating the frequency
# nltk.download('punkt_tab')

In [10]:
#Importing data from .csv (the train.csv file in this case)

texts  = pd.read_csv('Data/train/train.csv')
texts.head()


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [11]:
#grouping by author
byAuthor = texts.groupby('author')

#frequency by author
wordFreqByAuthor = nltk.probability.ConditionalFreqDist()

for name, group in byAuthor:
    '''
    transforming all the sentences from an author into one string
    helps us get the frequency of a word
    '''
    sentence = group['text'].str.cat(sep = ' ')

    '''
    lowering the string so we don't encounter issues if
    words are written with an capital letter or not
    '''
    sentence = sentence.lower()

    #splitting the sentence into words(tokens)
    tokens = nltk.tokenize.word_tokenize(sentence)

    #caluclate the frequency of every word(token)
    frequency = nltk.FreqDist(tokens)

    #add the frequency for each author to our dictionary
    wordFreqByAuthor[name] = frequency


In [16]:
#test
testSentence = "It was a dark and stormy night."

#tokenizing our sentence
preProcessedTestSentence = nltk.tokenize.word_tokenize(testSentence.lower())

#create an empty DataFrame
testProbabilities = pd.DataFrame(columns = ['author','word','probability'])

#iterating trough every author
for i in wordFreqByAuthor.keys():
    #iterating trough every word of the test sentence
    for j in preProcessedTestSentence:
        #frequency of the respective word
        wordFreq = wordFreqByAuthor[i].freq(j)
        #adding a small amount for every probability so none is 0
        smoothedWordFreq = wordFreq + 0.00001

        #making an aux variable output so we can append it in the result variable we want
        output = pd.DataFrame([[i,j,smoothedWordFreq]], columns = ['author','word','probability'])
        testProbabilities = pd.concat([testProbabilities,output], ignore_index = True)

#making a data frame where we calculate the probability of all the authors using the same word
testProbabilitiesByAuthors = pd.DataFrame(columns = ['author','jointProbability'])

for i in wordFreqByAuthor.keys():
    #get the joint probability that each author wrote each word
    oneAuthor = testProbabilities[testProbabilities['author'] == i ]
    #calculates the probability of one author using the respective word in comparison to all the other authors
    jointProbability = oneAuthor['probability'].prod()

    #add it to the DataFrame
    output = pd.DataFrame([[i,jointProbability]], columns = ['author','jointProbability'])
    testProbabilitiesByAuthors = pd.concat([testProbabilitiesByAuthors,output], ignore_index = True)

testProbabilitiesByAuthors.loc[testProbabilitiesByAuthors['jointProbability'].idxmax(),'author']


C:\Users\Andrei\AppData\Local\Temp\ipykernel_2436\4277797762.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testProbabilities = pd.concat([testProbabilities,output], ignore_index = True)
C:\Users\Andrei\AppData\Local\Temp\ipykernel_2436\4277797762.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  testProbabilitiesByAuthor = pd.concat([testProbabilitiesByAuthor,output], ignore_index = True)


'HPL'